## Install Requirements and Import files

In [ ]:
!git clone https://github.com/BilkentCeleste/ReMediCard.io.git
!mv /content/ReMediCard.io/generation/* /content/
!pip install -r requirements.txt

# Initiliaze Credentials

In [ ]:
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE
CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE CREDENTIALS HERE

## Load STT Model

In [ ]:
import whisper
from queue_service import RedisQueue
from storage_service import S3_Service
from generation_service import GenerationService

model = whisper.load_model("medium")
queue = RedisQueue(queue_name=GENERATION_QUEUE_NAME, host=HOST, db=0, password=PASSWORD)
s3Service = S3_Service(AWS_ACCESS_KEY, AWS_SECRET_KEY, REGION)
generationService = GenerationService(API_ENDPOINT, API_KEY)

In [ ]:
import json

message = queue.wait_message(
    queue_name=VOICE_QUEUE_NAME,
    host=HOST,
    port=PORT,
    db=0,
    password=PASSWORD
)

message = json.loads(message[1])
S3_URL = message["address"]
LANGUAGE = message["language"]

s3Service.download_from_s3_url(
    s3_url=S3_URL,
    local_path=LOCAL_PATH
)

audio_file = "/content/downloads/sound.mp3"

language = "en" if LANGUAGE == "ENGLISH" else "tr"

result = model.transcribe(audio_file, language=language)  # "tr" is for Turkish

print(result["text"])

prompt = "Generate an array of flashcards(not specifically should be in basic question forms try to form complex front and back pairs) in the form of an array of json objects for the following text(which is a video transcript for an educative video), don't add any other comment or just I want just an array of json objects with fields front and back so the answer must be like[{\"back\":"",\"front\":"",}], here is the text: " + result["text"]
response = generationService.generate_query(prompt)

response_text = response["candidates"][0]["content"]["parts"][0]["text"]

first_squared_bracket = response_text.find("[")
last_squared_bracket = response_text.rfind("]") + 1

deck = json.loads(response_text[first_squared_bracket:last_squared_bracket])

deckTask = {"flashcards": deck, "name": message["fileName"], "userId": message["userId"]}
queue.enqueue(deckTask)

length = queue.get_queue_length()